In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils
from sklearn.metrics import confusion_matrix, f1_score, auc, roc_curve, precision_recall_curve

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# Language Model

In [5]:
lm = True
clean = False

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/tmp_lm'),
 WindowsPath('../input/train.csv')]

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
allText_df = train_df.copy().append(test_df.copy(), sort=False)

In [7]:
if clean:
    train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
    test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x:clean_numbers(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

#train_x = train_df['question_text'].fillna('_##_').values
#test_x = test_df['question_text'].fillna('_##_').values

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [8]:
#print(df_train.head(3))

print(train_df.shape)
print(test_df.shape)

In [9]:
# df_train.head(3)

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = path/'train.csv',
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT)] 
    test = torchtext.data.TabularDataset( 
                path=path/'test.csv',
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


LABEL.batch_first


In [12]:
TEXT.build_vocab(train, test, vectors = "glove.6B.50d")

In [13]:
train_storage = train

In [252]:
#train = train_storage

In [253]:
train,_ = train.split(0.1)

In [254]:
from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size= 64, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)

test_iter = Iterator(
    test,
    batch_size = 64,
    sort = False,
    sort_within_batch = False,
    repeat = False)

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [255]:
#iter(train_iter).__next__().question_text
len(train_iter)
label_Size = next(iter(train_iter)).target
len(label_Size[label_Size==1])
#train_iter.batches
#train_iter = iter(train_iter)
#train_iter.__next__().question_text
#train_iter.__next__().target
#iter(train_iter).__next__().question_text.shape

2

In [256]:
#smaller,_ = train.split(0.1)

In [257]:
#smaller.examples[0].question_text

In [258]:
#len(train)
len(test_iter)

5872

In [259]:
#train.examples[0].question_text

In [260]:
TEXT.vocab.vectors.shape

torch.Size([527626, 50])

TEXT.vocab.freqs

In [312]:
embed_size = 50
hidden_size = 150
ver1 = False
sigBCE = True
bidir = True
layer_LSTM = 2
layer_Lin = 3
dropout_LSTM = 0.1
dropout_Lin = 0.1

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        
        hidden_size = 64
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size*4,2)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        #print(train.shape)
        h_embedding = self.embedding(train)
        #print(h_embedding.shape)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
        #print(h_embedding.shape)
        
        h_lstm, _ = self.lstm(h_embedding)
        #print(h_lstm.shape)
        h_gru, _ = self.gru(h_lstm)
        #print(h_gru.shape)
        
        avg_pool = torch.mean(h_gru,1)
        #print(avg_pool.shape)
        #avg_pool = avg_pool.view(-1,64)
        max_pool, _ = torch.max(h_gru,1)
        #print(max_pool.shape)
        
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        #print(conc.shape)
        #conc = self.dropout(conc)
        print(conc.shape)
        #out = nn.Sigmoid()(conc)
        
        out = nn.LogSoftmax(dim=1)(conc)
        
        #out = self.out(conc)
        #print(out[:,-1].shape)
        
        return out
    
    
    
    
batch = next(iter(train_iter)).question_text
net = NeuralNet()
#net(batch)

In [262]:
#batch = next(iter(train_iter)).question_text
#batch[30]
#layer_Lin

In [313]:
class LSTMTagger(nn.Module):
    def __init__(self):
        super(LSTMTagger, self).__init__()
                      
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        
        
        #self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm_Layer = nn.LSTM(embed_size, hidden_size,num_layers = layer_LSTM, dropout = dropout_LSTM, bidirectional = bidir)
        
        self.lin_Layer = []
        
        self.lin_Layer_Dropout = dropout_Lin
        
        if bidir:
            scale = 2
        else:
            scale = 1
        
        for _ in range(layer_Lin - 1):
            self.lin_Layer.append(nn.Linear(hidden_size*scale,hidden_size*scale))
            self.lin_Layer = nn.ModuleList(self.lin_Layer)
        if sigBCE:
            self.hidden2tag = nn.Linear(hidden_size*scale, 1)
        else:
            self.hidden2tag = nn.Linear(hidden_size*scale, 2)
    
    def forward(self, train):
        #print(train.shape)
        embeds = self.embedding(train)
        #print(embeds.shape)
        #print(embeds.view(len(train),64,-1).shape)
        #print(embeds.view(len(train),train.shape[1],-1).shape)
        lstm_out, _ = self.lstm_Layer(embeds)#embeds.view(len(train),train.shape[1],-1))
        #print(lstm_out.shape)
        
        if ver1:
            #print(lstm_out.shape)
            feature = lstm_out[-1,:,:] # -1 grabs last layer in list
            #print(feature.shape)
        
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                predict = self.hidden2tag(feature)
        
            feature = F.relu(feature)
            predict = self.hidden2tag(feature)
            if sigBCE:
                return nn.Sigmoid()(predict)
            else:
                return nn.LogSoftmax(dim=1)(predict)
        else:
           
            avg_pool = torch.mean(lstm_out,0)
            #print(avg_pool.shape)
            target_space = self.hidden2tag(avg_pool)#.view(len(train),-1))
            #print(target_space.shape)
            if sigBCE:
                return nn.Sigmoid()(target_space)
            else:
                return F.log_softmax(target_space, dim=1)
            #print(target_score.shape)

inputs_check = next(iter(train_iter)).question_text

#print(inputs_check.shape)

model = LSTMTagger()

with torch.no_grad():
    inputs = inputs_check
    #print(len(inputs))
    score = model(inputs)
    print(score.shape)
    print(score)

torch.Size([64, 1])
tensor([[0.5038],
        [0.5027],
        [0.5037],
        [0.5036],
        [0.5037],
        [0.5022],
        [0.5034],
        [0.5029],
        [0.5016],
        [0.5011],
        [0.5035],
        [0.5013],
        [0.5041],
        [0.5030],
        [0.5035],
        [0.5039],
        [0.5040],
        [0.5035],
        [0.5035],
        [0.5021],
        [0.5024],
        [0.5024],
        [0.5026],
        [0.5030],
        [0.5039],
        [0.5030],
        [0.5037],
        [0.5033],
        [0.5035],
        [0.5038],
        [0.5016],
        [0.5026],
        [0.5037],
        [0.5026],
        [0.5029],
        [0.5017],
        [0.5030],
        [0.5020],
        [0.5009],
        [0.5031],
        [0.5036],
        [0.5035],
        [0.5030],
        [0.5035],
        [0.5033],
        [0.5027],
        [0.5011],
        [0.5034],
        [0.5038],
        [0.5022],
        [0.5031],
        [0.5032],
        [0.5035],
        [0.5013],
        

In [314]:
def weight_init(m):
    if  isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

In [315]:
model = LSTMTagger()
model.apply(weight_init)
if sigBCE:
    loss_function = nn.BCELoss(reduction='none')
else:
    loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.2, momentum=0.9)

In [316]:
validate = True

checking = next(iter(train_iter))
inputs_check = checking.question_text
target_check = checking.target


f1_train = []
auc_train = []
train_accu = []

avg_loss = 0

with torch.no_grad():
    inputs = inputs_check
    score1 = model(inputs)
    #print(score)
    
for epoch in range(10):
    model.train()
    avg_loss = 0
    training_set = iter(train_iter)
    model.zero_grad()
    all_preds = []
    all_actual = []
    for batch_num in range(len(train_iter)):
        
        sentence_data = next(training_set)
        sentence_in = sentence_data.question_text
        #target -> target.unsqueeze(1)
        if sigBCE:
            targets = sentence_data.target.float()#.unsqueeze(1)
        else:
            targets = torch.LongTensor(sentence_data.target)#.unsqueeze(1)
        optimizer.zero_grad()
        
        target_scores = model(sentence_in)
        
        loss = loss_function(target_scores.squeeze(1), targets)#.squeeze(1))#.reshape(64,1).squeeze(1))
        loss = (loss * (targets*14+1)/(targets*14+1).sum()).sum()#.mean()
        loss.backward()
        
        if sigBCE:
            prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
            accuracy = np.asarray(prediction == np.asarray(targets)).sum()/len(targets)*100
        else:
            prediction = target_scores.data.max(1)[1]
            accuracy = np.asarray(prediction==targets).sum()/len(targets)*100
        train_accu.append(accuracy)
        
        if sigBCE:
            all_preds.extend(target_scores.tolist())#torch.round(target_scores).tolist())
        else:
            all_preds.extend(torch.exp(target_scores).tolist())#torch.round(target_scores).tolist())
        all_actual.extend(targets.tolist())
        
        #print(np.asarray(target_scores)[:,1].max())
        
        #targets -> targets.reshape(64,1).squeeze(1)
        
        avg_loss += loss.item()
        
        #print(loss)
        if sigBCE:
            print(batch_num+1,"out of: ",len(train_iter), " With Loss of: ",loss.item()," Acc: ",accuracy, " Best Score of: ", np.asarray(target_scores).max())
        else:
            print(batch_num+1,"out of: ",len(train_iter), " With Loss of: ",loss.item()," Acc: ",accuracy, " Best Score of: ", np.asarray(torch.exp(target_scores))[:,1].max())
        
        optimizer.step()
    
    #model.eval()
    avg_loss = avg_loss/len(train_iter)
    
    #if validate:
        #avg_val
    
    
    #else:
    print("Training loss for epoch {} is {}".format(epoch + 1, avg_loss))
    
    
    #train_confusion_matrix = confusion_matrix(all_actual, all_preds).ravel()
    if sigBCE:
        pred_Clas = (np.asarray(all_preds) > 0.5).astype(int)#-2.65).astype(int)#
    else:
        pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)#-2.65).astype(int)#
    train_confusion_matrix = confusion_matrix(all_actual, pred_Clas).ravel()
    
    print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
    f1_train.append(f1_score(all_actual, pred_Clas))
    print("F1 score for epoch {} is {}".format(epoch + 1, f1_train[-1]))
    fpr, tpr, _ = roc_curve(all_actual, pred_Clas, pos_label=1)
    auc_train.append(auc(fpr, tpr))
    print("AUC for epoch {} is {}\n".format(epoch + 1, auc_train[-1]))

with torch.no_grad():
    inputs = inputs_check
    score2 = model(inputs)
    #print(score)
        
        

1 out of:  2041  With Loss of:  0.7078242897987366  Acc:  1.5625  Best Score of:  0.53529346
2 out of:  2041  With Loss of:  0.6793862581253052  Acc:  95.3125  Best Score of:  0.49622568
3 out of:  2041  With Loss of:  0.7397185564041138  Acc:  90.625  Best Score of:  0.45170102
4 out of:  2041  With Loss of:  0.6710286140441895  Acc:  95.3125  Best Score of:  0.45772377
5 out of:  2041  With Loss of:  0.7044904232025146  Acc:  90.625  Best Score of:  0.46176317
6 out of:  2041  With Loss of:  0.7219658493995667  Acc:  90.625  Best Score of:  0.5096081
7 out of:  2041  With Loss of:  0.6641263365745544  Acc:  73.4375  Best Score of:  0.5970915
8 out of:  2041  With Loss of:  0.6130262613296509  Acc:  84.375  Best Score of:  0.65311146
9 out of:  2041  With Loss of:  0.6637488007545471  Acc:  90.625  Best Score of:  0.6803183
10 out of:  2041  With Loss of:  0.6823036670684814  Acc:  84.375  Best Score of:  0.71590024
11 out of:  2041  With Loss of:  0.7127294540405273  Acc:  70.3125  B

88 out of:  2041  With Loss of:  0.5991429686546326  Acc:  67.1875  Best Score of:  0.83838075
89 out of:  2041  With Loss of:  0.6968387365341187  Acc:  62.5  Best Score of:  0.86916745
90 out of:  2041  With Loss of:  0.6687873601913452  Acc:  50.0  Best Score of:  0.8883598
91 out of:  2041  With Loss of:  0.6300121545791626  Acc:  48.4375  Best Score of:  0.87979513
92 out of:  2041  With Loss of:  0.5952639579772949  Acc:  48.4375  Best Score of:  0.87692165
93 out of:  2041  With Loss of:  0.6341680884361267  Acc:  59.375  Best Score of:  0.86297137
94 out of:  2041  With Loss of:  0.6974425315856934  Acc:  73.4375  Best Score of:  0.89352936
95 out of:  2041  With Loss of:  0.6679660081863403  Acc:  57.8125  Best Score of:  0.8825971
96 out of:  2041  With Loss of:  0.5764116048812866  Acc:  53.125  Best Score of:  0.9046061
97 out of:  2041  With Loss of:  0.6145943403244019  Acc:  79.6875  Best Score of:  0.85516363
98 out of:  2041  With Loss of:  0.5814437866210938  Acc:  82

174 out of:  2041  With Loss of:  0.739393413066864  Acc:  56.25  Best Score of:  0.86976063
175 out of:  2041  With Loss of:  0.4852377772331238  Acc:  79.6875  Best Score of:  0.95925146
176 out of:  2041  With Loss of:  0.3704402446746826  Acc:  93.75  Best Score of:  0.90120935
177 out of:  2041  With Loss of:  0.44643446803092957  Acc:  95.3125  Best Score of:  0.8994443
178 out of:  2041  With Loss of:  0.5330154299736023  Acc:  90.625  Best Score of:  0.8463084
179 out of:  2041  With Loss of:  0.5425804853439331  Acc:  90.625  Best Score of:  0.9729417
180 out of:  2041  With Loss of:  0.7584200501441956  Acc:  76.5625  Best Score of:  0.9990049
181 out of:  2041  With Loss of:  0.6397473216056824  Acc:  70.3125  Best Score of:  0.9765286
182 out of:  2041  With Loss of:  0.7980847954750061  Acc:  54.6875  Best Score of:  0.9990798
183 out of:  2041  With Loss of:  0.7678173184394836  Acc:  51.5625  Best Score of:  0.9983524
184 out of:  2041  With Loss of:  0.758500337600708  

260 out of:  2041  With Loss of:  0.4994264841079712  Acc:  84.375  Best Score of:  0.9821862
261 out of:  2041  With Loss of:  0.629545271396637  Acc:  73.4375  Best Score of:  0.99884987
262 out of:  2041  With Loss of:  0.592332124710083  Acc:  64.0625  Best Score of:  0.9997265
263 out of:  2041  With Loss of:  0.4294728636741638  Acc:  68.75  Best Score of:  0.99533963
264 out of:  2041  With Loss of:  0.4726932644844055  Acc:  64.0625  Best Score of:  0.99976665
265 out of:  2041  With Loss of:  0.7706127166748047  Acc:  67.1875  Best Score of:  0.9888636
266 out of:  2041  With Loss of:  0.5069707036018372  Acc:  78.125  Best Score of:  0.97740763
267 out of:  2041  With Loss of:  0.6662675738334656  Acc:  92.1875  Best Score of:  0.65820074
268 out of:  2041  With Loss of:  0.6456685066223145  Acc:  95.3125  Best Score of:  0.6947252
269 out of:  2041  With Loss of:  0.4146828055381775  Acc:  98.4375  Best Score of:  0.77422595
270 out of:  2041  With Loss of:  0.47748354077339

346 out of:  2041  With Loss of:  0.4532528519630432  Acc:  84.375  Best Score of:  0.9423546
347 out of:  2041  With Loss of:  0.297214537858963  Acc:  92.1875  Best Score of:  0.9839901
348 out of:  2041  With Loss of:  0.44197866320610046  Acc:  79.6875  Best Score of:  0.94339585
349 out of:  2041  With Loss of:  0.42947643995285034  Acc:  84.375  Best Score of:  0.9702804
350 out of:  2041  With Loss of:  0.6615301966667175  Acc:  90.625  Best Score of:  0.7933766
351 out of:  2041  With Loss of:  0.3158470094203949  Acc:  87.5  Best Score of:  0.9610365
352 out of:  2041  With Loss of:  0.35899680852890015  Acc:  90.625  Best Score of:  0.99977785
353 out of:  2041  With Loss of:  0.26283663511276245  Acc:  84.375  Best Score of:  0.9999367
354 out of:  2041  With Loss of:  0.4863319396972656  Acc:  85.9375  Best Score of:  0.9862629
355 out of:  2041  With Loss of:  0.45385533571243286  Acc:  82.8125  Best Score of:  0.9995271
356 out of:  2041  With Loss of:  0.3135332763195038

...

1723 out of:  2041  With Loss of:  0.32173454761505127  Acc:  90.625  Best Score of:  0.7504225
1724 out of:  2041  With Loss of:  0.3230234384536743  Acc:  87.5  Best Score of:  0.98789227
1725 out of:  2041  With Loss of:  0.276406466960907  Acc:  87.5  Best Score of:  0.9720427
1726 out of:  2041  With Loss of:  0.17483267188072205  Acc:  93.75  Best Score of:  0.99208814
1727 out of:  2041  With Loss of:  0.5995581746101379  Acc:  79.6875  Best Score of:  0.9818039
1728 out of:  2041  With Loss of:  0.31172409653663635  Acc:  81.25  Best Score of:  0.99598396
1729 out of:  2041  With Loss of:  0.22092880308628082  Acc:  89.0625  Best Score of:  0.9915406
1730 out of:  2041  With Loss of:  0.127454474568367  Acc:  93.75  Best Score of:  0.9988494
1731 out of:  2041  With Loss of:  0.2386837601661682  Acc:  90.625  Best Score of:  0.9415292
1732 out of:  2041  With Loss of:  0.5613163113594055  Acc:  90.625  Best Score of:  0.95031446
1733 out of:  2041  With Loss of:  0.512669563293

1808 out of:  2041  With Loss of:  0.205017551779747  Acc:  89.0625  Best Score of:  0.97486365
1809 out of:  2041  With Loss of:  0.25979265570640564  Acc:  87.5  Best Score of:  0.9845333
1810 out of:  2041  With Loss of:  0.3345794081687927  Acc:  81.25  Best Score of:  0.9994174
1811 out of:  2041  With Loss of:  0.7470130324363708  Acc:  87.5  Best Score of:  0.9992576
1812 out of:  2041  With Loss of:  0.2737415134906769  Acc:  87.5  Best Score of:  0.9609328
1813 out of:  2041  With Loss of:  0.44008100032806396  Acc:  85.9375  Best Score of:  0.99996185
1814 out of:  2041  With Loss of:  0.31319424510002136  Acc:  87.5  Best Score of:  0.97188276
1815 out of:  2041  With Loss of:  0.282145619392395  Acc:  81.25  Best Score of:  0.9446564
1816 out of:  2041  With Loss of:  0.21684867143630981  Acc:  82.8125  Best Score of:  0.999218
1817 out of:  2041  With Loss of:  0.15965887904167175  Acc:  92.1875  Best Score of:  0.93991804
1818 out of:  2041  With Loss of:  0.2086805105209

1893 out of:  2041  With Loss of:  0.2694421410560608  Acc:  85.9375  Best Score of:  0.9970849
1894 out of:  2041  With Loss of:  0.16992607712745667  Acc:  95.3125  Best Score of:  0.994662
1895 out of:  2041  With Loss of:  0.7516838312149048  Acc:  85.9375  Best Score of:  0.99113
1896 out of:  2041  With Loss of:  0.27539539337158203  Acc:  85.9375  Best Score of:  0.98620737
1897 out of:  2041  With Loss of:  0.4828408360481262  Acc:  87.5  Best Score of:  0.94333285
1898 out of:  2041  With Loss of:  0.2133992463350296  Acc:  90.625  Best Score of:  0.8497585
1899 out of:  2041  With Loss of:  0.20213693380355835  Acc:  90.625  Best Score of:  0.9925006
1900 out of:  2041  With Loss of:  0.23647834360599518  Acc:  89.0625  Best Score of:  0.96296954
1901 out of:  2041  With Loss of:  0.2133413553237915  Acc:  95.3125  Best Score of:  0.942036
1902 out of:  2041  With Loss of:  0.3314899206161499  Acc:  84.375  Best Score of:  0.8792079
1903 out of:  2041  With Loss of:  0.394419

1978 out of:  2041  With Loss of:  0.2658059298992157  Acc:  82.8125  Best Score of:  0.97333413
1979 out of:  2041  With Loss of:  0.1747596114873886  Acc:  92.1875  Best Score of:  0.98525965
1980 out of:  2041  With Loss of:  0.2329908311367035  Acc:  87.5  Best Score of:  0.9426183
1981 out of:  2041  With Loss of:  0.20922401547431946  Acc:  85.9375  Best Score of:  0.98061407
1982 out of:  2041  With Loss of:  0.2919278144836426  Acc:  87.5  Best Score of:  0.95999
1983 out of:  2041  With Loss of:  0.6034626960754395  Acc:  82.8125  Best Score of:  0.90729773
1984 out of:  2041  With Loss of:  0.2139451503753662  Acc:  84.375  Best Score of:  0.99688834
1985 out of:  2041  With Loss of:  0.32357800006866455  Acc:  76.5625  Best Score of:  0.99871683
1986 out of:  2041  With Loss of:  0.24416986107826233  Acc:  87.5  Best Score of:  0.97430104
1987 out of:  2041  With Loss of:  0.2055397629737854  Acc:  90.625  Best Score of:  0.9683155
1988 out of:  2041  With Loss of:  0.177622

KeyboardInterrupt: 

In [322]:
torch.save(model,"C:/Users/xw5735/PWinFolder/WPy-3661/notebooks/KaggleChal/LSTMTagger.pt")

C:\Users\xw5735\PWinFolder\WPy-3661\python-3.6.6.amd64\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
batch_size = 10
nb_classes = 2
target = torch.LongTensor(batch_size).random_(nb_classes)
#print((targets*5+1))
print((loss * (targets*5+1)).mean())

In [310]:
prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
target_scores

tensor([[0.5147],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5152],
        [0.5148],
        [0.5145],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5148],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5145],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#train_confusion_matrix = confusion_matrix(all_actual, preddd_clas).ravel()
#print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
#pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#f1_score(all_actual, pred_Clas)
#pred_Clas

In [ ]:
preddd = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
len(preddd[preddd>0])

In [ ]:
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result

In [ ]:
def threshold_search_2(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [ ]:
#threshold_search_2(all_actual, preddd)
np.asarray(all_preds)[:,1].max()

In [ ]:
preddd = np.asarray(all_preds)[:,1]
preddd_clas = (preddd>-0.45).astype(int)
preddd.max()
len(preddd_clas[preddd_clas == 1])

In [ ]:
thr = threshold_search_2(all_actual, np.asarray(all_preds));thr

In [ ]:
thr = threshold_search(all_actual, np.asarray(all_preds), plot=True);thr

In [323]:
all_final_preds = []
all_final_rounding = []
model.eval()
testing_batches = iter(test_iter)
for batch_num in range(len(test_iter)):
    print(batch_num+1,"out of: ",len(test_iter))
    final_preds = model(next(testing_batches).question_text)
    prediction = np.asarray(target_scores>0.5,dtype=float)
    all_final_preds.extend(final_preds.tolist())
    all_final_rounding.extend(prediction.tolist())
    

#probab = np.asarray(final_preds)[:,1]

#final_preds_clas = (probab>thr['threshold']).astype(int)

#final_preds_clas.shape

1 out of:  5872
2 out of:  5872
3 out of:  5872
4 out of:  5872
5 out of:  5872
6 out of:  5872
7 out of:  5872
8 out of:  5872
9 out of:  5872
10 out of:  5872
11 out of:  5872
12 out of:  5872
13 out of:  5872
14 out of:  5872
15 out of:  5872
16 out of:  5872
17 out of:  5872
18 out of:  5872
19 out of:  5872
20 out of:  5872
21 out of:  5872
22 out of:  5872
23 out of:  5872
24 out of:  5872
25 out of:  5872
26 out of:  5872
27 out of:  5872
28 out of:  5872
29 out of:  5872
30 out of:  5872
31 out of:  5872
32 out of:  5872
33 out of:  5872
34 out of:  5872
35 out of:  5872
36 out of:  5872
37 out of:  5872
38 out of:  5872
39 out of:  5872
40 out of:  5872
41 out of:  5872
42 out of:  5872
43 out of:  5872
44 out of:  5872
45 out of:  5872
46 out of:  5872
47 out of:  5872
48 out of:  5872
49 out of:  5872
50 out of:  5872
51 out of:  5872
52 out of:  5872
53 out of:  5872
54 out of:  5872
55 out of:  5872
56 out of:  5872
57 out of:  5872
58 out of:  5872
59 out of:  5872
60 out

462 out of:  5872
463 out of:  5872
464 out of:  5872
465 out of:  5872
466 out of:  5872
467 out of:  5872
468 out of:  5872
469 out of:  5872
470 out of:  5872
471 out of:  5872
472 out of:  5872
473 out of:  5872
474 out of:  5872
475 out of:  5872
476 out of:  5872
477 out of:  5872
478 out of:  5872
479 out of:  5872
480 out of:  5872
481 out of:  5872
482 out of:  5872
483 out of:  5872
484 out of:  5872
485 out of:  5872
486 out of:  5872
487 out of:  5872
488 out of:  5872
489 out of:  5872
490 out of:  5872
491 out of:  5872
492 out of:  5872
493 out of:  5872
494 out of:  5872
495 out of:  5872
496 out of:  5872
497 out of:  5872
498 out of:  5872
499 out of:  5872
500 out of:  5872
501 out of:  5872
502 out of:  5872
503 out of:  5872
504 out of:  5872
505 out of:  5872
506 out of:  5872
507 out of:  5872
508 out of:  5872
509 out of:  5872
510 out of:  5872
511 out of:  5872
512 out of:  5872
513 out of:  5872
514 out of:  5872
515 out of:  5872
516 out of:  5872
517 out of

917 out of:  5872
918 out of:  5872
919 out of:  5872
920 out of:  5872
921 out of:  5872
922 out of:  5872
923 out of:  5872
924 out of:  5872
925 out of:  5872
926 out of:  5872
927 out of:  5872
928 out of:  5872
929 out of:  5872
930 out of:  5872
931 out of:  5872
932 out of:  5872
933 out of:  5872
934 out of:  5872
935 out of:  5872
936 out of:  5872
937 out of:  5872
938 out of:  5872
939 out of:  5872
940 out of:  5872
941 out of:  5872
942 out of:  5872
943 out of:  5872
944 out of:  5872
945 out of:  5872
946 out of:  5872
947 out of:  5872
948 out of:  5872
949 out of:  5872
950 out of:  5872
951 out of:  5872
952 out of:  5872
953 out of:  5872
954 out of:  5872
955 out of:  5872
956 out of:  5872
957 out of:  5872
958 out of:  5872
959 out of:  5872
960 out of:  5872
961 out of:  5872
962 out of:  5872
963 out of:  5872
964 out of:  5872
965 out of:  5872
966 out of:  5872
967 out of:  5872
968 out of:  5872
969 out of:  5872
970 out of:  5872
971 out of:  5872
972 out of

1352 out of:  5872
1353 out of:  5872
1354 out of:  5872
1355 out of:  5872
1356 out of:  5872
1357 out of:  5872
1358 out of:  5872
1359 out of:  5872
1360 out of:  5872
1361 out of:  5872
1362 out of:  5872
1363 out of:  5872
1364 out of:  5872
1365 out of:  5872
1366 out of:  5872
1367 out of:  5872
1368 out of:  5872
1369 out of:  5872
1370 out of:  5872
1371 out of:  5872
1372 out of:  5872
1373 out of:  5872
1374 out of:  5872
1375 out of:  5872
1376 out of:  5872
1377 out of:  5872
1378 out of:  5872
1379 out of:  5872
1380 out of:  5872
1381 out of:  5872
1382 out of:  5872
1383 out of:  5872
1384 out of:  5872
1385 out of:  5872
1386 out of:  5872
1387 out of:  5872
1388 out of:  5872
1389 out of:  5872
1390 out of:  5872
1391 out of:  5872
1392 out of:  5872
1393 out of:  5872
1394 out of:  5872
1395 out of:  5872
1396 out of:  5872
1397 out of:  5872
1398 out of:  5872
1399 out of:  5872
1400 out of:  5872
1401 out of:  5872
1402 out of:  5872
1403 out of:  5872
1404 out of:

1783 out of:  5872
1784 out of:  5872
1785 out of:  5872
1786 out of:  5872
1787 out of:  5872
1788 out of:  5872
1789 out of:  5872
1790 out of:  5872
1791 out of:  5872
1792 out of:  5872
1793 out of:  5872
1794 out of:  5872
1795 out of:  5872
1796 out of:  5872
1797 out of:  5872
1798 out of:  5872
1799 out of:  5872
1800 out of:  5872
1801 out of:  5872
1802 out of:  5872
1803 out of:  5872
1804 out of:  5872
1805 out of:  5872
1806 out of:  5872
1807 out of:  5872
1808 out of:  5872
1809 out of:  5872
1810 out of:  5872
1811 out of:  5872
1812 out of:  5872
1813 out of:  5872
1814 out of:  5872
1815 out of:  5872
1816 out of:  5872
1817 out of:  5872
1818 out of:  5872
1819 out of:  5872
1820 out of:  5872
1821 out of:  5872
1822 out of:  5872
1823 out of:  5872
1824 out of:  5872
1825 out of:  5872
1826 out of:  5872
1827 out of:  5872
1828 out of:  5872
1829 out of:  5872
1830 out of:  5872
1831 out of:  5872
1832 out of:  5872
1833 out of:  5872
1834 out of:  5872
1835 out of:

...

3938 out of:  5872
3939 out of:  5872
3940 out of:  5872
3941 out of:  5872
3942 out of:  5872
3943 out of:  5872
3944 out of:  5872
3945 out of:  5872
3946 out of:  5872
3947 out of:  5872
3948 out of:  5872
3949 out of:  5872
3950 out of:  5872
3951 out of:  5872
3952 out of:  5872
3953 out of:  5872
3954 out of:  5872
3955 out of:  5872
3956 out of:  5872
3957 out of:  5872
3958 out of:  5872
3959 out of:  5872
3960 out of:  5872
3961 out of:  5872
3962 out of:  5872
3963 out of:  5872
3964 out of:  5872
3965 out of:  5872
3966 out of:  5872
3967 out of:  5872
3968 out of:  5872
3969 out of:  5872
3970 out of:  5872
3971 out of:  5872
3972 out of:  5872
3973 out of:  5872
3974 out of:  5872
3975 out of:  5872
3976 out of:  5872
3977 out of:  5872
3978 out of:  5872
3979 out of:  5872
3980 out of:  5872
3981 out of:  5872
3982 out of:  5872
3983 out of:  5872
3984 out of:  5872
3985 out of:  5872
3986 out of:  5872
3987 out of:  5872
3988 out of:  5872
3989 out of:  5872
3990 out of:

4369 out of:  5872
4370 out of:  5872
4371 out of:  5872
4372 out of:  5872
4373 out of:  5872
4374 out of:  5872
4375 out of:  5872
4376 out of:  5872
4377 out of:  5872
4378 out of:  5872
4379 out of:  5872
4380 out of:  5872
4381 out of:  5872
4382 out of:  5872
4383 out of:  5872
4384 out of:  5872
4385 out of:  5872
4386 out of:  5872
4387 out of:  5872
4388 out of:  5872
4389 out of:  5872
4390 out of:  5872
4391 out of:  5872
4392 out of:  5872
4393 out of:  5872
4394 out of:  5872
4395 out of:  5872
4396 out of:  5872
4397 out of:  5872
4398 out of:  5872
4399 out of:  5872
4400 out of:  5872
4401 out of:  5872
4402 out of:  5872
4403 out of:  5872
4404 out of:  5872
4405 out of:  5872
4406 out of:  5872
4407 out of:  5872
4408 out of:  5872
4409 out of:  5872
4410 out of:  5872
4411 out of:  5872
4412 out of:  5872
4413 out of:  5872
4414 out of:  5872
4415 out of:  5872
4416 out of:  5872
4417 out of:  5872
4418 out of:  5872
4419 out of:  5872
4420 out of:  5872
4421 out of:

4800 out of:  5872
4801 out of:  5872
4802 out of:  5872
4803 out of:  5872
4804 out of:  5872
4805 out of:  5872
4806 out of:  5872
4807 out of:  5872
4808 out of:  5872
4809 out of:  5872
4810 out of:  5872
4811 out of:  5872
4812 out of:  5872
4813 out of:  5872
4814 out of:  5872
4815 out of:  5872
4816 out of:  5872
4817 out of:  5872
4818 out of:  5872
4819 out of:  5872
4820 out of:  5872
4821 out of:  5872
4822 out of:  5872
4823 out of:  5872
4824 out of:  5872
4825 out of:  5872
4826 out of:  5872
4827 out of:  5872
4828 out of:  5872
4829 out of:  5872
4830 out of:  5872
4831 out of:  5872
4832 out of:  5872
4833 out of:  5872
4834 out of:  5872
4835 out of:  5872
4836 out of:  5872
4837 out of:  5872
4838 out of:  5872
4839 out of:  5872
4840 out of:  5872
4841 out of:  5872
4842 out of:  5872
4843 out of:  5872
4844 out of:  5872
4845 out of:  5872
4846 out of:  5872
4847 out of:  5872
4848 out of:  5872
4849 out of:  5872
4850 out of:  5872
4851 out of:  5872
4852 out of:

5231 out of:  5872
5232 out of:  5872
5233 out of:  5872
5234 out of:  5872
5235 out of:  5872
5236 out of:  5872
5237 out of:  5872
5238 out of:  5872
5239 out of:  5872
5240 out of:  5872
5241 out of:  5872
5242 out of:  5872
5243 out of:  5872
5244 out of:  5872
5245 out of:  5872
5246 out of:  5872
5247 out of:  5872
5248 out of:  5872
5249 out of:  5872
5250 out of:  5872
5251 out of:  5872
5252 out of:  5872
5253 out of:  5872
5254 out of:  5872
5255 out of:  5872
5256 out of:  5872
5257 out of:  5872
5258 out of:  5872
5259 out of:  5872
5260 out of:  5872
5261 out of:  5872
5262 out of:  5872
5263 out of:  5872
5264 out of:  5872
5265 out of:  5872
5266 out of:  5872
5267 out of:  5872
5268 out of:  5872
5269 out of:  5872
5270 out of:  5872
5271 out of:  5872
5272 out of:  5872
5273 out of:  5872
5274 out of:  5872
5275 out of:  5872
5276 out of:  5872
5277 out of:  5872
5278 out of:  5872
5279 out of:  5872
5280 out of:  5872
5281 out of:  5872
5282 out of:  5872
5283 out of:

5662 out of:  5872
5663 out of:  5872
5664 out of:  5872
5665 out of:  5872
5666 out of:  5872
5667 out of:  5872
5668 out of:  5872
5669 out of:  5872
5670 out of:  5872
5671 out of:  5872
5672 out of:  5872
5673 out of:  5872
5674 out of:  5872
5675 out of:  5872
5676 out of:  5872
5677 out of:  5872
5678 out of:  5872
5679 out of:  5872
5680 out of:  5872
5681 out of:  5872
5682 out of:  5872
5683 out of:  5872
5684 out of:  5872
5685 out of:  5872
5686 out of:  5872
5687 out of:  5872
5688 out of:  5872
5689 out of:  5872
5690 out of:  5872
5691 out of:  5872
5692 out of:  5872
5693 out of:  5872
5694 out of:  5872
5695 out of:  5872
5696 out of:  5872
5697 out of:  5872
5698 out of:  5872
5699 out of:  5872
5700 out of:  5872
5701 out of:  5872
5702 out of:  5872
5703 out of:  5872
5704 out of:  5872
5705 out of:  5872
5706 out of:  5872
5707 out of:  5872
5708 out of:  5872
5709 out of:  5872
5710 out of:  5872
5711 out of:  5872
5712 out of:  5872
5713 out of:  5872
5714 out of:

sub = pd.read_csv(path/'sample_submission.csv')
sub['prediction'] = np.asarray(all_final_preds)>0.5
sub.to_csv("submission_Kag.csv",index=False)

In [ ]:
thr['threshold']

def train_model(model, train_data, epochs, show=False):
    
    #optimizer = torch.optim.Adam(model.parameters())
    optimizer = optim.SGD(model.parameters(), lr=0.0001)
    #loss_function = nn.BCEWithLogitsLoss(reduction='mean')
    loss_function = nn.NLLLoss()
    
    print("The loss function being used is {}".format(loss_function))
    errors = []
    eval_errors = []
    f1_train = []
    auc_train = []
    f1_eval = []
    auc_eval = []
    
    num_training_batches = len(train_data)
    
    for epoch in range(epochs):
        print("Epoch {}".format(epoch + 1))
        print("Training mode")
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        for batch_num in range(num_training_batches):
            print("Batch {}".format(batch_num + 1))
            print("Out of {}".format(num_training_batches))
            batch = next(train_iter)
            text, class_vector = batch.question_text.transpose(0,1),batch.target.unsqueeze(1)
            #print(batch)
            #print(text.shape)
            #optimizer.zero_grad()
            text_pred = model(text)
            #print(text_pred)
            print(class_vector.shape)
            print(torch.max(class_vector,1)[0].shape)
            loss = loss_function(text_pred,class_vector.reshape(64,1).squeeze(1)) #torch.max(class_vector,1)[0])
            print(loss)
            print(text_pred.squeeze(1))
            print(torch.max(class_vector,1)[0])
            loss.backward()
            optimizer.step()
    

    
    

In [ ]:
#train_iters = iter(train_iter)

In [ ]:
#wee = next(train_iters)

In [ ]:
#wee.question_text.shape